In [ ]:
!pip install -Uqq datasets seacrowd

!mkdir IDASVspoofing_train
!mkdir IDASVspoofing_val

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 14.3 MB/s eta 0:00:00
   ━━━━

In [ ]:
from datasets import load_dataset
import pandas as pd
import librosa
import soundfile as sf
from IPython.display import Audio, display
from tqdm.auto import tqdm

In [ ]:
dataset = load_dataset("SEACrowd/indspeech_news_lvcsr", trust_remote_code=True)

Generating train split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/daa6300c2ec826ec5479d23eafb597d64b8d6369d21f6575f2b76ea905a0ef24/Ind001/Ind001_F_B_C_news_0002.wav


Generating test split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/f62af82344f80d54ce9ab0714185a08811db444921a1769a092a9978ee541c3a/Ind239/Ind239_M_J_C_news_1348.wav


# Training & Validation

In [ ]:
# See sample dataset information
train_data = dataset['train']
for i, item in enumerate(train_data):
  if i == 3:
    break

  print(item)

{'spk_id': 'Ind001', 'gender': 'F', 'accent': 'B', 'type': 'C', 'txt_id': '0002', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/daa6300c2ec826ec5479d23eafb597d64b8d6369d21f6575f2b76ea905a0ef24/Ind001/Ind001_F_B_C_news_0002.wav', 'array': array([-9.15527344e-05, -4.27246094e-04, -5.18798828e-04, ...,
       -1.22070312e-04,  0.00000000e+00,  6.10351562e-05]), 'sampling_rate': 16000}, 'text': 'IMF bukan penguasa.'}
{'spk_id': 'Ind001', 'gender': 'F', 'accent': 'B', 'type': 'C', 'txt_id': '0025', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/daa6300c2ec826ec5479d23eafb597d64b8d6369d21f6575f2b76ea905a0ef24/Ind001/Ind001_F_B_C_news_0025.wav', 'array': array([ 0.00015259,  0.        , -0.00018311, ..., -0.00021362,
       -0.00027466,  0.00015259]), 'sampling_rate': 16000}, 'text': 'Syahril Syabirin.'}
{'spk_id': 'Ind001', 'gender': 'F', 'accent': 'B', 'type': 'C', 'txt_id': '0015', 'audio': {'path': '/root/.cache/huggingface/datasets/dow

In [ ]:
# Get the training split (you can also use 'validation' or 'test')
train_data = dataset['train'].shuffle(seed=42)

# Create a list to store the information
data_list = []

# Iterate through the dataset to get text and audio file info
for i, item in enumerate(train_data):
  train_split = int(0.8 * len(train_data))

  if i < train_split:
    split = 'T'
  else:
    split = 'V'

  data_dict = {
      'file_id': f"ID_{split}_{item['spk_id']}_{item['gender']}_{item['txt_id']}",
      'spk_id': f"ID_{item['spk_id']}",
      'text': item['text'],
      'audio_file': item['audio']['path'],
      'audio_duration': item['audio']['array'].shape[0] / item['audio']['sampling_rate']  # Duration in seconds
  }
  data_list.append(data_dict)

# Convert to pandas DataFrame for easier viewing
df = pd.DataFrame(data_list)

# Display basic statistics
print("Dataset Statistics:")
print(f"Total number of samples: {len(df)}")
print(f"Average audio duration: {df['audio_duration'].mean():.2f} seconds")

# Optional: Save to CSV
df.to_csv('indspeech_news_lvcsr_train_val.csv', index=False)

Dataset Statistics:
Total number of samples: 39600
Average audio duration: 3.52 seconds


In [ ]:
!mv /content/indspeech_news_lvcsr_train_val.csv /content/drive/MyDrive/dataspoofv2

In [ ]:
df.head(20)

,file_id,spk_id,text,audio_file,audio_duration
0,ID_T_Ind058_M_1973,ID_Ind058,Kami akan tetap di sini sampai Eurico dibebaskan.,/root/.cache/huggingface/datasets/downloads/ex...,4.34
1,ID_T_Ind073_F_2412,ID_Ind073,Akbar mengiyakan.,/root/.cache/huggingface/datasets/downloads/ex...,2.68
2,ID_T_Ind358_F_1940,ID_Ind358,Salah satunya dialami oleh pabrik sepatu merek...,/root/.cache/huggingface/datasets/downloads/ex...,4.49
3,ID_T_Ind161_F_2021,ID_Ind161,Sebagai ungkapan syukur.,/root/.cache/huggingface/datasets/downloads/ex...,2.65
4,ID_T_Ind152_M_1710,ID_Ind152,Bukan perang etnis.,/root/.cache/huggingface/datasets/downloads/ex...,2.32
5,ID_T_Ind319_F_0618,ID_Ind319,Pada kedua partai catur cepat itu.,/root/.cache/huggingface/datasets/downloads/ex...,3.31
6,ID_T_Ind214_M_0444,ID_Ind214,Dengan sistem proteksi yang sederhana berupa t...,/root/.cache/huggingface/datasets/downloads/ex...,5.30
7,ID_T_Ind210_F_0362,ID_Ind210,Privatisasi Bermasalah karena Pemerintah Tak P...,/root/.cache/huggingface/datasets/downloads/ex...,7.43
8,ID_T_Ind186_F_2897,ID_Ind186,Ujar Soetandyo.,/root/.cache/huggingface/datasets/downloads/ex...,2.48
9,ID_T_Ind263_F_2079,ID_Ind263,Ditangkap di bandara Internasional Ninoy Aquino.,/root/.cache/huggingface/datasets/downloads/ex...,4.10


In [ ]:
# Calculate the number of female and male samples
female_count = df[df['file_id'].str.contains('F')]['file_id'].count()
male_count = df[df['file_id'].str.contains('M')]['file_id'].count()

print("TRAINING INFORMATION")
print(f"Number of female samples: {female_count}")
print(f"Number of male samples: {male_count}")
print(f"Number of unique speakers: {len(set(dataset['train']['spk_id']))}")

TRAINING INFORMATION
Number of female samples: 19800
Number of male samples: 19800
Number of unique speakers: 360


In [ ]:
# Split dataframe to train and validation dataframes
train_mask = df['file_id'].str.contains('T')
val_mask = df['file_id'].str.contains('V')

train_df = df[train_mask]
val_df = df[val_mask]

In [ ]:
# Calculate the number of female and male samples
female_count = train_df[train_df['file_id'].str.contains('F')]['file_id'].count()
male_count = train_df[train_df['file_id'].str.contains('M')]['file_id'].count()

print("TRAIN DATASET INFORMATION")
print(f"Number of female samples: {female_count}")
print(f"Number of male samples: {male_count}")
print(f"Number of unique speakers: {len(set(dataset['train']['spk_id']))}")

TRAIN DATASET INFORMATION
Number of female samples: 15834
Number of male samples: 15846
Number of unique speakers: 360


In [ ]:
# Calculate the number of female and male samples
female_count = val_df[val_df['file_id'].str.contains('F')]['file_id'].count()
male_count = val_df[val_df['file_id'].str.contains('M')]['file_id'].count()

print("VALIDATION DATASET INFORMATION")
print(f"Number of female samples: {female_count}")
print(f"Number of male samples: {male_count}")
print(f"Number of unique speakers: {len(set(dataset['train']['spk_id']))}")

VALIDATION DATASET INFORMATION
Number of female samples: 3966
Number of male samples: 3954
Number of unique speakers: 360


In [ ]:
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_bonafide
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_bonafide/flac
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_bonafide
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_bonafide/flac

In [ ]:
male_df = train_df[train_df['file_id'].str.contains('M')].index

protocol_lines = []
counter = 0
for index, row in train_df.iloc[male_df].iterrows():
  if counter == 2500:
    break

  line = f"{row['spk_id']} {row['file_id']} - - bonafide"
  audio, sr = librosa.load(row['audio_file'], sr=None)
  sf.write(f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_bonafide/flac/{row['file_id']}.flac", audio, sr, format='FLAC')
  protocol_lines.append(line)

  counter += 1

female_df = train_df[train_df['file_id'].str.contains('F')].index
counter = 0
for index, row in train_df.iloc[female_df].iterrows():
  if counter == 2500:
    break

  line = f"{row['spk_id']} {row['file_id']} - - bonafide"
  audio, sr = librosa.load(row['audio_file'], sr=None)
  sf.write(f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_bonafide/flac/{row['file_id']}.flac", audio, sr, format='FLAC')
  protocol_lines.append(line)

  counter += 1

with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.train.bonafide.txt', 'w') as f:
  f.write('\n'.join(protocol_lines))

In [ ]:
val_df = val_df.reset_index(drop=True)
male_df = val_df[val_df['file_id'].str.contains('M')].index

protocol_lines = []
counter = 0
for index, row in val_df.iloc[male_df].iterrows():
  if counter == 1500:
    break

  line = f"{row['spk_id']} {row['file_id']} - - bonafide"
  audio, sr = librosa.load(row['audio_file'], sr=None)
  sf.write(f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_bonafide/flac/{row['file_id']}.flac", audio, sr, format='FLAC')
  protocol_lines.append(line)

  counter += 1

female_df = val_df[val_df['file_id'].str.contains('F')].index
counter = 0
for index, row in val_df.iloc[female_df].iterrows():
  if counter == 1500:
    break

  line = f"{row['spk_id']} {row['file_id']} - - bonafide"
  audio, sr = librosa.load(row['audio_file'], sr=None)
  sf.write(f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_bonafide/flac/{row['file_id']}.flac", audio, sr, format='FLAC')
  protocol_lines.append(line)

  counter += 1

with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.val.bonafide.txt', 'w') as f:
  f.write('\n'.join(protocol_lines))

In [ ]:
# !zip -r /content/IDASVspoofing_train.zip /content/IDASVspoofing_train
# !zip -r /content/IDASVspoofing_val.zip /content/IDASVspoofing_val

# from google.colab import files
# files.download("/content/IDASVspoofing_train.zip")
# files.download("/content/IDASVspoofing_val.zip")
# files.download("/content/IDASVspoofing.cm.train.txt")
# files.download("/content/IDASVspoofing.cm.val.txt")

Streaming output truncated to the last 5000 lines.
  adding: content/IDASVspoofing_train/flac/ID_T_Ind072_M_2347.flac (stored 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind054_M_1766.flac (deflated 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind058_M_1953.flac (stored 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind048_M_1641.flac (deflated 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind009_F_0338.flac (deflated 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind005_F_0177.flac (stored 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind074_M_2427.flac (deflated 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind007_M_0260.flac (deflated 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind043_F_1419.flac (deflated 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind068_M_2220.flac (deflated 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind069_M_2276.flac (stored 0%)
  adding: content/IDASVspoofing_train/flac/ID_T_Ind041_F_1358.flac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Testing

In [ ]:
test_data = dataset['test']

# Create a list to store the information
data_list = []

# Iterate through the dataset to get text and audio file info
for i, item in enumerate(test_data):

  data_dict = {
      'file_id': f"ID_TT_{item['spk_id']}_{item['gender']}_{item['txt_id']}",
      'spk_id': f"ID_{item['spk_id']}",
      'text': item['text'],
      'audio_file': item['audio']['path'],
      'audio_duration': item['audio']['array'].shape[0] / item['audio']['sampling_rate']  # Duration in seconds
  }
  data_list.append(data_dict)

# Convert to pandas DataFrame for easier viewing
df = pd.DataFrame(data_list)

# Display basic statistics
print("Dataset Statistics:")
print(f"Total number of samples: {len(df)}")
print(f"Average audio duration: {df['audio_duration'].mean():.2f} seconds")

# Optional: Save to CSV
df.to_csv('indspeech_news_lvcsr_test.csv', index=False)

Dataset Statistics:
Total number of samples: 4400
Average audio duration: 3.80 seconds


In [ ]:
!mv /content/indspeech_news_lvcsr_test.csv /content/drive/MyDrive/dataspoofv2

In [ ]:
# Split dataframe to train and validation dataframes
test_mask = df['file_id'].str.contains('TT')
test_df = df[test_mask]
test_df.head(20)

,file_id,spk_id,text,audio_file,audio_duration
0,ID_TT_Ind239_M_1348,ID_Ind239,Mingguan Survei Ekonomi Timur Tengah.,/root/.cache/huggingface/datasets/downloads/ex...,3.79
1,ID_TT_Ind239_M_1254,ID_Ind239,Baik swasta maupun negeri.,/root/.cache/huggingface/datasets/downloads/ex...,3.34
2,ID_TT_Ind239_M_1355,ID_Ind239,Marwah Daud Jenguk Akbar.,/root/.cache/huggingface/datasets/downloads/ex...,3.06
3,ID_TT_Ind239_M_1270,ID_Ind239,Jika ia merasa dirugikan oleh Tyasno.,/root/.cache/huggingface/datasets/downloads/ex...,4.92
4,ID_TT_Ind239_M_1276,ID_Ind239,Selain bajaj.,/root/.cache/huggingface/datasets/downloads/ex...,2.27
5,ID_TT_Ind239_M_1284,ID_Ind239,Kasus ini terungkap setelah polisi mengintai Y...,/root/.cache/huggingface/datasets/downloads/ex...,5.39
6,ID_TT_Ind239_M_1344,ID_Ind239,Kalau brankas bengkel dibawa kabur perampok.,/root/.cache/huggingface/datasets/downloads/ex...,4.41
7,ID_TT_Ind239_M_1330,ID_Ind239,Hal tersebut disampaikan Editor Asia Observer.,/root/.cache/huggingface/datasets/downloads/ex...,5.20
8,ID_TT_Ind239_M_1338,ID_Ind239,Sebelum periode Cacuk.,/root/.cache/huggingface/datasets/downloads/ex...,3.05
9,ID_TT_Ind239_M_1306,ID_Ind239,Apa enggak ngawur itu?,/root/.cache/huggingface/datasets/downloads/ex...,2.94


In [ ]:
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_bonafide
!mkdir /content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_bonafide/flac

mkdir: cannot create directory ‘/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_bonafide’: File exists


In [ ]:
male_df = test_df[test_df['file_id'].str.contains('M')].index

protocol_lines = []
counter = 0
for index, row in test_df.iloc[male_df].iterrows():
  if counter == 1000:
    break

  line = f"{row['spk_id']} {row['file_id']} - - bonafide"
  audio, sr = librosa.load(row['audio_file'], sr=None)
  sf.write(f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_bonafide/flac/{row['file_id']}.flac", audio, sr, format='FLAC')
  protocol_lines.append(line)

  counter += 1

female_df = test_df[test_df['file_id'].str.contains('F')].index
counter = 0
for index, row in test_df.iloc[female_df].iterrows():
  if counter == 1000:
    break

  line = f"{row['spk_id']} {row['file_id']} - - bonafide"
  audio, sr = librosa.load(row['audio_file'], sr=None)
  sf.write(f"/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_bonafide/flac/{row['file_id']}.flac", audio, sr, format='FLAC')
  protocol_lines.append(line)

  counter += 1

with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.bonafide.txt', 'w') as f:
  f.write('\n'.join(protocol_lines))

In [ ]:
# !zip -r /content/IDASVspoofing_test.zip /content/IDASVspoofing_test

# from google.colab import files
# files.download("/content/IDASVspoofing_test.zip")
# files.download("/content/IDASVspoofing.cm.test.txt")

  adding: content/IDASVspoofing_test/ (stored 0%)
  adding: content/IDASVspoofing_test/flac/ (stored 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind381_F_2676.flac (stored 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind341_M_1422.flac (deflated 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind341_M_1410.flac (deflated 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind342_M_1422.flac (deflated 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind354_F_1777.flac (deflated 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind239_M_1280.flac (deflated 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind250_M_1683.flac (stored 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind335_M_1157.flac (deflated 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind239_M_1269.flac (stored 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind350_F_1637.flac (stored 0%)
  adding: content/IDASVspoofing_test/flac/ID_TT_Ind355_F_1849.flac (deflated 0%)
  adding: co

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
print(len(os.listdir('/content/drive/MyDrive/dataspoofv2/IDASVspoofing_train_bonafide/flac')))
print(len(os.listdir('/content/drive/MyDrive/dataspoofv2/IDASVspoofing_val_bonafide/flac')))
print(len(os.listdir('/content/drive/MyDrive/dataspoofv2/IDASVspoofing_test_bonafide/flac')))

5000
3000
2000


In [ ]:
with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.train.bonafide.txt', 'r') as file:
   line_count = sum(1 for line in file)
print(f"Number of lines: {line_count}")

with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.val.bonafide.txt', 'r') as file:
   line_count = sum(1 for line in file)
print(f"Number of lines: {line_count}")

with open('/content/drive/MyDrive/dataspoofv2/IDASVspoofing.cm.test.bonafide.txt', 'r') as file:
   line_count = sum(1 for line in file)
print(f"Number of lines: {line_count}")

Number of lines: 5000
Number of lines: 3000
Number of lines: 2000


In [ ]:
type(train_data)

datasets.arrow_dataset.Dataset

In [ ]:
train_data.to_csv("/content/drive/MyDrive/dataspoofv2/trainval_bonafide_shuffle.csv")

Creating CSV from Arrow format:   0%|          | 0/40 [00:00<?, ?ba/s]

9260460